In [0]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

# The unique characters in the file
path_to_file ='gdrive/My Drive/HAIKU/data.tsv'
text = open(path_to_file).read()
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

for char,_ in zip(char2idx, range(20)):
    print('{:6s} ---> {:4d}'.format(repr(char), char2idx[char]))
    


96 unique characters
'\n'   --->    0
' '    --->    1
'!'    --->    2
'"'    --->    3
'#'    --->    4
'$'    --->    5
'%'    --->    6
'&'    --->    7
"'"    --->    8
'('    --->    9
')'    --->   10
'*'    --->   11
'+'    --->   12
','    --->   13
'-'    --->   14
'.'    --->   15
'/'    --->   16
'0'    --->   17
'1'    --->   18
'2'    --->   19


In [0]:
for char,_ in zip(char2idx, range(100)):
    print('{:6s} ---> {:4d}'.format(repr(char), char2idx[char]))
    


'\n'   --->    0
' '    --->    1
'!'    --->    2
'"'    --->    3
'#'    --->    4
'$'    --->    5
'%'    --->    6
'&'    --->    7
"'"    --->    8
'('    --->    9
')'    --->   10
'*'    --->   11
'+'    --->   12
','    --->   13
'-'    --->   14
'.'    --->   15
'/'    --->   16
'0'    --->   17
'1'    --->   18
'2'    --->   19
'3'    --->   20
'4'    --->   21
'5'    --->   22
'6'    --->   23
'7'    --->   24
'8'    --->   25
'9'    --->   26
':'    --->   27
';'    --->   28
'<'    --->   29
'='    --->   30
'>'    --->   31
'?'    --->   32
'@'    --->   33
'A'    --->   34
'B'    --->   35
'C'    --->   36
'D'    --->   37
'E'    --->   38
'F'    --->   39
'G'    --->   40
'H'    --->   41
'I'    --->   42
'J'    --->   43
'K'    --->   44
'L'    --->   45
'M'    --->   46
'N'    --->   47
'O'    --->   48
'P'    --->   49
'Q'    --->   50
'R'    --->   51
'S'    --->   52
'T'    --->   53
'U'    --->   54
'V'    --->   55
'W'    --->   56
'X'    --->   57
'Y'    --->   

In [0]:
 #The maximum length sentence we want for a single input in characters
seq_length = 60

# Create training examples / targets
chunks = tf.data.Dataset.from_tensor_slices(text_as_int).batch(seq_length+1, drop_remainder=True)

for item in chunks.take(5):
  print(repr(''.join(idx2char[item.numpy()])))



"Can't you see how much<br> better you make the world just<br>"
"by being in it?\nI'm fine. I'm listening<br> to music and eati"
"ng lunch.<br>You're still an asshole.\nYour cat has no more<br"
'> metaphysical value<br>than a deer or cow.\nThis only makes m'
'e<br> wonder where else our lovely<br>adventure can go.\nThat '


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = chunks.map(split_input_target)
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))



Input data:  "Can't you see how much<br> better you make the world just<br"
Target data: "an't you see how much<br> better you make the world just<br>"


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 36 ('C')
  expected output: 66 ('a')
Step    1
  input: 66 ('a')
  expected output: 79 ('n')
Step    2
  input: 79 ('n')
  expected output: 8 ("'")
Step    3
  input: 8 ("'")
  expected output: 85 ('t')
Step    4
  input: 85 ('t')
  expected output: 1 (' ')


In [0]:
# Batch size 
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 2000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [0]:
class Model(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, units):
    super(Model, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    if tf.test.is_gpu_available():
      self.gru = tf.keras.layers.CuDNNGRU(self.units, 
                                          return_sequences=True, 
                                          recurrent_initializer='glorot_uniform',
                                          stateful=True)
    else:
      self.gru = tf.keras.layers.LSTM(self.units, 
                                     return_sequences=True, 
                                     recurrent_activation='sigmoid', 
                                     recurrent_initializer='glorot_uniform', 
                                     stateful=True)

    self.fc = tf.keras.layers.Dense(vocab_size)
        
  def call(self, x):
    embedding = self.embedding(x)
    
    # output at every time step
    # output shape == (batch_size, seq_length, hidden_size) 
    output = self.gru(embedding)
    
    
    # The dense layer will output predictions for every time_steps(seq_length)
    # output shape after the dense layer == (seq_length * batch_size, vocab_size)
    prediction = self.fc(output)
    
    # states will be used to pass at every step to the model while training
    return prediction

In [0]:
!cat data.tsv

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
units = 512

model = Model(vocab_size, embedding_dim, units)


In [0]:
# Using adam optimizer with default arguments
optimizer = tf.train.AdamOptimizer()

# Using sparse_softmax_cross_entropy so that we don't have to create one-hot vectors
def loss_function(real, preds):
    return tf.losses.sparse_softmax_cross_entropy(labels=real, logits=preds)


In [0]:
model.build(tf.TensorShape([BATCH_SIZE, seq_length]))


In [0]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      multiple                  24576     
_________________________________________________________________
cu_dnngru_7 (CuDNNGRU)       multiple                  1182720   
_________________________________________________________________
dense_8 (Dense)              multiple                  49248     
Total params: 1,256,544
Trainable params: 1,256,544
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")


In [0]:
EPOCHS =15


In [0]:
# Training loop
for epoch in range(EPOCHS):
    start = time.time()
    
    # initializing the hidden state at the start of every epoch
    # initally hidden is None
    hidden = model.reset_states()
    
    for (batch, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              # feeding the hidden state back into the model
              # This is the interesting step
              predictions = model(inp)
              loss = loss_function(target, predictions)
              
          grads = tape.gradient(loss, model.variables)
          optimizer.apply_gradients(zip(grads, model.variables))

          if batch % 100 == 0:
              print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch+1,
                                                            batch,
                                                            loss))
    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix)

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 4.5631
Epoch 1 Batch 100 Loss 2.2776
Epoch 1 Batch 200 Loss 2.0869
Epoch 1 Batch 300 Loss 1.9096
Epoch 1 Batch 400 Loss 1.7601
Epoch 1 Batch 500 Loss 1.7199
Epoch 1 Batch 600 Loss 1.6204
Epoch 1 Batch 700 Loss 1.5018
Epoch 1 Batch 800 Loss 1.5158
Epoch 1 Batch 900 Loss 1.5423
Epoch 1 Batch 1000 Loss 1.4891
Epoch 1 Batch 1100 Loss 1.4964
Epoch 1 Batch 1200 Loss 1.4918
Epoch 1 Batch 1300 Loss 1.4381
Epoch 1 Batch 1400 Loss 1.4198
Epoch 1 Batch 1500 Loss 1.4315
Epoch 1 Batch 1600 Loss 1.3753
Epoch 1 Batch 1700 Loss 1.4284
Epoch 1 Batch 1800 Loss 1.3684
Epoch 1 Batch 1900 Loss 1.3885
Epoch 1 Batch 2000 Loss 1.3582
Epoch 1 Batch 2100 Loss 1.3558
Epoch 1 Batch 2200 Loss 1.3363
Epoch 1 Batch 2300 Loss 1.3198
Epoch 1 Batch 2400 Loss 1.2835
Epoch 1 Batch 2500 Loss 1.3313
Epoch 1 Batch 2600 Loss 1.2945
Epoch 1 Batch 2700 Loss 1.3301
Epoch 1 Batch 2800 Loss 1.3174
Epoch 1 Batch 2900 Loss 1.3111
Epoch 1 Batch 3000 Loss 1.2875
Epoch 1 Batch 3100 Loss 1.3949
Epoch 1 Batch 3200 L

KeyboardInterrupt: ignored

In [0]:
model.save_weights(checkpoint_prefix)


In [0]:
!ls {checkpoint_dir}


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt'

In [0]:
model = Model(vocab_size, embedding_dim, units)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


In [0]:
# Evaluation step (generating text using the learned model)

  # Number of characters to generate
num_generate = 12
# You can change the start string to experiment
start_string = 'I failed'

# Converting our start string to numbers (vectorizing) 
input_eval = [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)

# Empty string to store our results
text_generated = []

# Low temperatures results in more predictable text.
# Higher temperatures results in more surprising text.
# Experiment to find the best setting.
temperature = 0.9

# Evaluation loop.

# Here batch size == 1
model.reset_states()
count_line=0

while ((count_line<2) or (not( idx2char[predicted_id]=='.'))):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a multinomial distribution to predict the word returned by the model
    predictions = predictions / temperature
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
    if predicted_id==29:
      count_line+=1;
      
    # We pass the predicted word as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)
    
    text_generated.append(idx2char[predicted_id])

print ((start_string + ''.join(text_generated))[:])


I failed her.
No just to use<br> different as hell if you should<br>do anything? Why?
I read the one<br> has a fucking look or you<br>really seen these much.


I failed the same<br> and see what the same thing that's<br>about to see that.
The only really<br>
